# Assistente Virtual para Google Colab

## 1. Instalação das Dependências

In [ ]:
!pip install gTTS SpeechRecognition wikipedia ffmpeg-python

## 2. Código de Captura de Áudio do Navegador
Esta célula contém o código necessário para gravar áudio do seu microfone no navegador e passá-lo para o ambiente Python do Colab.

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("div");
var my_p = document.createElement("p");
var my_btn = document.createElement("button");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
  recordButton.innerText = "Recording... press to stop";
};

recordButton.onclick = function() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... please wait!";
  } else {
    navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);
  }
};
</script>
"""

def get_audio_from_browser():
  display(HTML(AUDIO_HTML))
  data = eval_js("base64data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True))

  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  return riff

## 3. Criação dos Módulos Python

In [ ]:
%%writefile speech.py
from gtts import gTTS
from IPython.display import Audio, display
import speech_recognition as sr

def speak(text, lang='pt-br'):
    """Converte texto em áudio e o exibe no Colab."""
    try:
        tts = gTTS(text=text, lang=lang, slow=False)
        audio_file = 'audio.mp3'
        tts.save(audio_file)
        print(f"Assistente: {text}")
        display(Audio(audio_file, autoplay=True))
    except Exception as e:
        print(f"Erro ao tentar falar: {e}")

# A função get_audio será diferente, usando o helper do Colab
# A definição final será no script principal do notebook

Overwriting speech.py


In [ ]:
%%writefile commands.py
import webbrowser
import wikipedia
import random
from datetime import datetime
from speech import speak

wikipedia.set_lang("pt")

def search_youtube(keyword):
    url = f"https://www.youtube.com/results?search_query={keyword}"
    print(f"Abra este link no seu navegador: {url}")
    speak(f"Aqui está o que eu encontrei para {keyword} no youtube")

def search_wikipedia(query):
    try:
        result = wikipedia.summary(query, sentences=3)
        speak("De acordo com a Wikipedia")
        print(result)
        speak(result)
    except wikipedia.exceptions.PageError:
        speak(f"Não encontrei nenhum resultado para {query} na Wikipedia.")
    except wikipedia.exceptions.DisambiguationError as e:
        speak(f"Existem múltiplos resultados para {query}. Por favor, seja mais específico.")

def get_joke():
    jokes = ["O que o pato disse para a pata? Vem quá!", "Qual é a fórmula da água benta? H Deus O!"]
    speak(random.choice(jokes))

def get_time():
    strTime = datetime.now().strftime("%H:%M")
    speak(f"São {strTime}")

def find_pharmacy():
    url = "https://www.google.com/maps/search/farmácias"
    print(f"Abra este link no seu navegador: {url}")
    speak("Mostrando o mapa com as farmácias mais próximas.")

Writing commands.py


## 4. Execução do Assistente Virtual

In [ ]:
import speech_recognition as sr
from speech import speak
from commands import search_youtube, search_wikipedia, get_joke, get_time, find_pharmacy
import io

def get_audio_colab():
    r = sr.Recognizer()
    audio_data = get_audio_from_browser()
    with io.BytesIO(audio_data) as f:
        audio = sr.AudioFile(f)
        with r as source:
            audio_record = r.record(audio)
    try:
        text = r.recognize_google(audio_record, language='pt-BR')
        print(f"Você disse: {text}")
        return text.lower()
    except sr.UnknownValueError:
        speak("Desculpe, não entendi.")
    except sr.RequestError:
        speak("Desculpe, o serviço não está disponível.")
    return ""

def run_assistant():
    speak("Olá! Como posso ajudar? Diga um comando.")
    text = get_audio_colab()
    if text:
        if 'youtube' in text:
            speak("O que você quer pesquisar no Youtube?")
            keyword = get_audio_colab()
            if keyword:
                search_youtube(keyword)
        elif 'pesquisar' in text:
            speak("O que você quer pesquisar?")
            query = get_audio_colab()
            if query:
                search_wikipedia(query)
        elif 'piada' in text:
            get_joke()
        elif 'que horas são' in text:
            get_time()
        elif 'farmácia' in text:
            find_pharmacy()
        else:
            speak("Comando não reconhecido.")

# Para executar, chame a função run_assistant() na célula abaixo

In [ ]:
run_assistant()

Assistente: Olá! Como posso ajudar? Diga um comando.


AttributeError: 'int' object has no attribute 'split'